### PKA-ENTITY-RESOLUTION

In [ ]:
import pyspark
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession

- This code imports necessary libraries and sets up the environment for PySpark.


In [ ]:
spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('chapter_2').getOrCreate()

 - This code initializes a SparkSession with specific configurations, setting the driver memory to 16 GB and specifying the application name as 'chapter_2'.


In [ ]:
prev = spark.read.csv("data/linkage/donation/block_1/block_1.csv")
prev

- This code reads a CSV file named "block_1.csv" located in the "data/linkage/donation/block_1/" directory into a DataFrame named `prev`.


In [ ]:
prev.show(2)

- This code displays the first two rows of the DataFrame `prev`.


In [ ]:
parsed = spark.read.option("header", "true").option("nullValue", "?").\
option("inferSchema", "true").csv("data/linkage/donation/block_1/
↪block_1.csv")

- This code reads a CSV file named "block_1.csv" located in the "data/linkage/donation/block_1/" directory into a DataFrame named `parsed`.
- It sets options such as treating the first row as the header, specifying "?" as the null value, and inferring the schema automatically.


In [ ]:
parsed.printSchema()
parsed.show(5)

- This code prints the schema of the DataFrame `parsed` and displays the first five rows of the DataFrame.


In [ ]:
parsed.count()

- This code counts the number of rows in the DataFrame `parsed`.


In [ ]:
parsed.cache()

- This code caches the DataFrame `parsed` into memory for faster access.


In [ ]:
from pyspark.sql.functions import col
parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

- This code groups the DataFrame `parsed` by the "is_match" column and counts the occurrences of each unique value.
- It then orders the results in descending order based on the count.
- Finally, it displays the results.


In [ ]:
parsed.createOrReplaceTempView("linkage")

- This code creates a temporary view named "linkage" for the DataFrame `parsed`, allowing SQL queries to be executed on it.


In [ ]:
spark.sql("""
SELECT is_match, COUNT(*) cnt
FROM linkage
GROUP BY is_match
ORDER BY cnt DESC
""").show()

- This code executes a SQL query using Spark SQL. It selects the "is_match" column and counts the occurrences of each unique value from the "linkage" temporary view.
- The results are then ordered in descending order based on the count.


In [ ]:
summary = parsed.describe()

- This code generates summary statistics for the DataFrame `parsed`.


In [ ]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

- This code selects specific columns ("summary", "cmp_fname_c1", "cmp_fname_c2") from the summary DataFrame generated earlier.


In [ ]:
matches = parsed.where("is_match = true")
match_summary = matches.describe()
misses = parsed.filter(col("is_match") == False)
miss_summary = misses.describe()

- This code filters the DataFrame `parsed` to obtain rows where the "is_match" column is equal to `true`, storing them in a new DataFrame called `matches`.
- It then generates summary statistics for the `matches` DataFrame and stores them in `match_summary`.
- Similarly, it filters `parsed` to obtain rows where the "is_match" column is equal to `false`, storing them in a new DataFrame called `misses`.
- It then generates summary statistics for the `misses` DataFrame and stores them in `miss_summary`.


In [ ]:
summary_p = summary.toPandas()

- This code converts the summary DataFrame `summary` to a Pandas DataFrame `summary_p`.


In [ ]:
summary_p.head()
summary_p.shape


- This code displays the first few rows of the Pandas DataFrame `summary_p`.
- It also shows the shape of the DataFrame.


In [ ]:
summary_p = summary_p.set_index('summary').transpose().reset_index()
summary_p = summary_p.rename(columns={'index':'field'})
summary_p = summary_p.rename_axis(None, axis=1)
summary_p.shape


- This code performs several transformations on the Pandas DataFrame `summary_p`:
  - It sets the index of the DataFrame to the 'summary' column, transposes the DataFrame, and resets the index.
  - It renames the 'index' column to 'field'.
  - It removes the axis name.
- Finally, it displays the shape of the transformed DataFrame.


In [ ]:
summaryT = spark.createDataFrame(summary_p)
summaryT


- This code creates a Spark DataFrame `summaryT` from the transformed Pandas DataFrame `summary_p`.


In [ ]:
summaryT.printSchema()


- This code prints the schema of the Spark DataFrame `summaryT`.


In [ ]:
from pyspark.sql.types import DoubleType

for c in summaryT.columns:
    if c == 'field':
        continue
    summaryT = summaryT.withColumn(c, summaryT[c].cast(DoubleType()))

summaryT.printSchema()


- This code iterates through the columns of the Spark DataFrame `summaryT`.
- For each column (except 'field'), it casts the values to DoubleType.


In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType

def pivot_summary(desc):
    # convert to pandas dataframe
    desc_p = desc.toPandas()
    # transpose
    desc_p = desc_p.set_index('summary').transpose().reset_index()
    desc_p = desc_p.rename(columns={'index':'field'})
    desc_p = desc_p.rename_axis(None, axis=1)
    # convert to Spark dataframe
    descT = spark.createDataFrame(desc_p)
    # convert metric columns to double from string
    for c in descT.columns:
        if c == 'field':
            continue
        else:
            descT = descT.withColumn(c, descT[c].cast(DoubleType()))
    return descT


- This code defines a function `pivot_summary` that takes a Spark DataFrame `desc` as input.
- It converts the input DataFrame to a Pandas DataFrame, transposes it, and converts it back to a Spark DataFrame.
- Then, it casts the metric columns to DoubleType.
- Finally, it returns the transformed Spark DataFrame.


In [ ]:
match_summaryT = pivot_summary(match_summary)
miss_summaryT = pivot_summary(miss_summary)


- This code applies the `pivot_summary` function to the `match_summary` and `miss_summary` DataFrames, transforming them into new DataFrames `match_summaryT` and `miss_summaryT`, respectively.


In [ ]:
match_summaryT.createOrReplaceTempView("match_desc")
miss_summaryT.createOrReplaceTempView("miss_desc")

spark.sql("""
SELECT a.field, a.count + b.count total, a.mean - b.mean delta
FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
WHERE a.field NOT IN ("id_1", "id_2")
ORDER BY delta DESC, total DESC
""")


- This code creates temporary views "match_desc" and "miss_desc" for the DataFrames `match_summaryT` and `miss_summaryT`, respectively.
- It then executes a SQL query that joins the two views on the "field" column, calculates the total count for each field, and computes the difference in mean values between matched and mismatched records.
- The query filters out fields "id_1" and "id_2" and orders the results by the difference in means in descending order, followed by the total count in descending order.


In [ ]:
good_features = ["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]
sum_expression = " + ".join(good_features)
sum_expression


- This code defines a list of good features named `good_features`.
- It then constructs a sum expression by joining the elements of `good_features` with the '+' operator.
